In [ ]:
from decimal import Decimal, getcontext
getcontext().prec = 32

In [ ]:
with open('../inputs/24.txt') as f:
    data = f.read().splitlines()

In [ ]:
def parse(lines):
    hailstones = []
    
    for line in lines:
        pos, vel = line.split(' @ ')
        hailstones.append((
            tuple(map(int, pos.split(', '))),
            tuple(map(int, vel.split(', '))),
        ))
        
    return hailstones

In [ ]:
# Part 1
def find_intersection(h1, h2):
    (x1, y1, _), (vx1, vy1, _) = h1
    (x2, y2, _), (vx2, vy2, _) = h2
    
    m1 = vy1 / vx1
    m2 = vy2 / vx2
    
    if m1 == m2:
        return

    c1 = y1 - (m1 * x1)
    c2 = y2 - (m2 * x2)
    
    x = (c2 - c1) / (m1 - m2)
    y = (m1 * x) + c1
    
    return (x, y)

def is_valid(hailstone, intersection):
    (px, py, _), (vx, vy, _) = hailstone
    ix, iy = intersection
    
    return not (
        (vx > 0 and px > ix) or 
        (vx < 0 and px < ix) or 
        (vy > 0 and py > iy) or 
        (vy < 0 and py < iy))

def count_hailstones(hailstones, min, max):
    total = 0

    for i, h1 in enumerate(hailstones):
        for j, h2 in enumerate(hailstones[i:]):        
            intersection = find_intersection(h1, h2)
            
            if not intersection:
                continue
            
            x, y = intersection
            
            if not (min <= x <= max and min <= y <= max):
                continue
            
            if not is_valid(h1, intersection) or not is_valid(h2, intersection):
                continue
                    
            total += 1
            
    return total

count_hailstones(parse(data), 200000000000000, 400000000000000)

In [ ]:
# Part 2

# Rock      (X, Y, Z), (dX, dY, dZ)
# Hailstone (x, y, z), (dx, dy, dz)

# X + tdX = x + tdx
# tdX = x + tdx - X
# tdX - tdx = x - X
# t = (x - X) / (dX - dx)
# (x - X) / (dX - dx) = (y - Y) / (dY - dy)
# (x - X)(dY - dy) = (y - Y)(dX - dx)
# xdY - xdy - XdY + Xdy = ydX - ydx - YdX + Ydx
# YdX - XdY = ydX - ydx + Ydx - xdY + xdy - Xdy

# x₁dy₁ - x₁dY - Xdy₁ - y₁dx₁ + y₁dX + Ydx₁ = x₂dy₂ - x₂dY - Xdy₂ - y₂dx₂ + y₂dX + Ydx₂

def gaussian_elimination(m):
    for i in range(len(m)):
        t = m[i][i]
        m[i] = [x / t for x in m[i]]
        
        for j in range(i + 1, len(m)):
            t = m[j][i]
            m[j] = [x - t * m[i][k] for k, x in enumerate(m[j])]
    
    for i in range(len(m) - 1, -1, -1):
        for j in range(i):
            t = m[j][i]
            m[j] = [x - t * m[i][k] for k, x in enumerate(m[j])]
    
    return m

def matrix(stones, xi, yi):
    m = []
    
    for s in stones:
        x = s[0][xi]
        y = s[0][yi]
        dx = s[1][xi]
        dy = s[1][yi]
        
        m.append([
            Decimal(-dy), 
            Decimal(dx),
            Decimal(y), 
            Decimal(-x), 
            Decimal(y) * Decimal(dx) - Decimal(x) * Decimal(dy) 
        ])

    return [
        [a - b for a, b in zip(r, m[-1])] for r in m[:4]
    ]

def calc_stone_val(hailstones):
    x, y, *_ = [row[-1] for row in gaussian_elimination(matrix(hailstones, 0, 1))]
    z, *_ = [row[-1] for row in gaussian_elimination(matrix(hailstones, 2, 1))]

    return round(x + y + z)

calc_stone_val(parse(data)[:5])